In [1]:
class IntegerValue:
    def __init__(self, name):
        self.values = {}

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(instance, None)

In [2]:
import weakref

In [5]:
class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(instance, None)

In [6]:
class Point:
    x = IntegerValue()

In [7]:
p = Point()

In [8]:
print(hex(id(p)))

0x106c1a8d0


In [9]:
p.x = 100.1

In [10]:
p.x

100

In [11]:
Point.x.values.keyrefs()

[<weakref at 0x106c3d1c0; to 'Point' at 0x106c1a8d0>]

In [12]:
del p

In [13]:
Point.x.values.keyrefs()

[]

In [14]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(id(instance), None)

In [15]:
class Point:
    x = IntegerValue()

    def __init__(self, x):
        self.x = x

    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x

In [16]:
p = Point(10.1)

In [17]:
p.x

10

In [18]:
p.x = 20.2

In [19]:
p.x

20

In [21]:
id(p), Point.x.values

(4408438160, {4408438160: 20})

In [22]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [24]:
p_id = id(p)

In [25]:
ref_count(p_id)

1

In [26]:
del p

In [27]:
ref_count(p_id)

0

In [28]:
Point.x.values

{4408438160: 20}

In [29]:
p = Point(10.1)

In [30]:
weak_p = weakref.ref(p)

In [31]:
weak_p

<weakref at 0x106c9fe70; to 'Point' at 0x106c89350>

In [32]:
ref_count(id(p))

1

In [33]:
del p

In [34]:
weak_p

<weakref at 0x106c9fe70; dead>

In [35]:
def obj_destroyed(obj):
    print(f'{obj} destroyed...')

In [36]:
p = Point(10.1)
w = weakref.ref(p, obj_destroyed)

In [37]:
del p

<weakref at 0x1066fe7a0; dead> destroyed...


In [44]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.values.get(id(instance))
            return value_tuple[1]
    
    def _remove_object(self, weak_ref):
        print(f'Removing dead entry for {weak_ref}')

In [45]:
class Point:
    x = IntegerValue()

In [46]:
p1 = Point()
p2 = Point()

In [47]:
p1.x, p2.x = 10.1, 20.2

In [48]:
p1.x, p2.x

(10, 20)

In [49]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [50]:
del p1

Removing dead entry for <weakref at 0x106cffa60; dead>


In [52]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.values.get(id(instance))
            return value_tuple[1]
    
    def _remove_object(self, weak_ref):
        reverse_lookup = [key for key, value in self.values.items() if value[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.values[key]

In [53]:
class Point:
    x = IntegerValue()

In [54]:
p = Point()

In [55]:
p.x = 10.1

In [56]:
p.x

10

In [57]:
Point.x.values

{4408492624: (<weakref at 0x106d0efc0; to 'Point' at 0x106c44250>, 10)}

In [58]:
del p

In [59]:
Point.x.values

{}

In [60]:
class Person:
    pass

In [61]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [62]:
hasattr(Person.__weakref__, '__get__')

True

In [63]:
hasattr(Person.__weakref__, '__set__')

True

In [64]:
p = Person()

In [65]:
hasattr(p, '__weakref__')

True

In [66]:
print(p.__weakref__)

None


In [67]:
w = weakref.ref(p)

In [68]:
print(p.__weakref__)

<weakref at 0x106d0fa60; to 'Person' at 0x1066f9210>


In [69]:
class Person:
    __slots__ = 'name', 

In [70]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name',),
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

In [71]:
p = Person()

In [72]:
hasattr(p, '__weakref__')

False

In [73]:
w = weakref.ref(p)

TypeError: cannot create weak reference to 'Person' object

In [75]:
class Person:
    __slots__ = 'name', '__weakref__'

In [76]:
p = Person()

In [77]:
weakref.ref(p)

<weakref at 0x106d9b100; to 'Person' at 0x1066dc760>

In [78]:
class ValidString:
    def __init__(self, min_length=0, max_length=255):
        self.data = {}
        self._min_length = min_length
        self._max_length = max_length

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError('Expected a string')
        if len(value) < self._min_length:
            raise ValueError(f'String must be at least {self._min_length} characters')
        if len(value) > self._max_length:
            raise ValueError(f'String must be at most {self._max_length} characters')
        self.data[id(instance)] = (weakref.ref(instance, self._remove_object), value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]
    
    def _remove_object(self, weak_ref):
        reverse_lookup = [key for key, value in self.data.items() if value[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.data[key]

In [79]:
class Person:
    __slots__ = '__weakref__',

    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)

    def __eq__(self, other):
        return (
            isinstance(other, Person) and
            self.first_name == other.first_name and
            self.last_name == other.last_name
        )
    
class BankAccount:
    __slots__ = '__weakref__',

    account_number = ValidString(5, 255)

    def __eq__(self, other):
        return (
            isinstance(other, BankAccount) and
            self.account_number == other.account_number
        )

In [80]:
p1 = Person()

In [81]:
p1.first_name = ""

ValueError: String must be at least 1 characters

In [82]:
p2 = Person()

Removing dead entry for <weakref at 0x10673b420; dead>


In [83]:
p1.first_name, p1.last_name = 'Guido', 'van Rossum'

In [84]:
p1.first_name, p1.last_name

('Guido', 'van Rossum')

In [85]:
p2.first_name, p2.last_name = 'Raymond', 'Hettinger'

In [86]:
b1, b2 = BankAccount(), BankAccount()

In [87]:
b1.account_number, b2.account_number = 'Savings', 'Checking'

In [88]:
p1.first_name, p2.last_name

('Guido', 'Hettinger')

In [89]:
p2.first_name, p2.last_name

('Raymond', 'Hettinger')

In [90]:
b1.account_number, b2.account_number

('Savings', 'Checking')

In [91]:
Person.first_name.data.values()

dict_values([(<weakref at 0x10689d350; to 'Person' at 0x1068244f0>, 'Guido'), (<weakref at 0x106c74860; to 'Person' at 0x10683ee60>, 'Raymond')])

In [92]:
BankAccount.account_number.data.values()

dict_values([(<weakref at 0x106c3eb60; to 'BankAccount' at 0x106840fd0>, 'Savings'), (<weakref at 0x10677e480; to 'BankAccount' at 0x1068410c0>, 'Checking')])

In [93]:
del p1
del p2
del b1
del b2

In [95]:
Person.first_name.data.values()

dict_values([(<weakref at 0x10689d350; to 'Person' at 0x1068244f0>, 'Guido')])

In [96]:
BankAccount.account_number.data.values()

dict_values([])

In [97]:
Person.first_name.data

{4404167920: (<weakref at 0x10689d350; to 'Person' at 0x1068244f0>, 'Guido')}

In [98]:
ref_count(4404167920)

2